In [1]:
import pandas as pd
import numpy as np


In [2]:
furnitures_tb = pd.read_csv("./data/furnitures.csv")
zones_tb = pd.read_csv("./data/zones_production.csv")
resources_tb = pd.read_csv("./data/resources.csv")

In [3]:
for col in ["Resources", "Quantity"]:
    furnitures_tb[col] = furnitures_tb[col].apply(lambda x: str(x).split(","))
furnitures_tb.head()

,Items,Type,Resources,Quantity,Builder Profession
0,Hay Wall,Wall,[Straw],[1],NaN
1,Cob Wall,Wall,"[Wood Rod, Clay]","[3, 1]",NaN
2,Wattle Wall,Wall,"[Wood Rod, Straw, Clay]","[4, 1, 1]",NaN
3,Wooden Wall,Wall,[Wood],[1],Carpentering
4,Lime Wall,Wall,[Limestone Block],[1],Architecture


In [4]:
for col in ["Uses","Consumes", "Produces"]:
    zones_tb[col] = zones_tb[col].apply(lambda x: x.split(", "))

In [5]:
zones_tb.head()

,Timestamp,Zone Name,Profession,Uses,Consumes,Produces
0,7/7/2022 19:20,Theatre,Acting,"[Theatre Stage, Wooden Bench]",[None],[None]
1,7/1/2022 16:52,Alchemists Workshop,Alchemy,"[Alchemist's Workshop, Cauldron, Chest, Ground...","[Dandellion, Echinacea, Lavender, Magic Potion...","[Health Potion, Magic Potion, Philosopher Ston..."
2,7/1/2022 18:04,Builders Guild,Architecture,"[Builder's Desk, Chest, Ground Storage, Store ...","[Stone Block, Clay, Rope]",[Advanced Repair Kit]
3,7/1/2022 16:59,Artists Workshop,Art,"[Art Station, Blast Furnace, Chest, Ground Sto...","[Bronze Ingot, Coal, Copper Ore, Firewood, Gol...","[Art, Bronze Statue, Golden Statue, Silver Sta..."
4,7/1/2022 17:01,Bakery,Baking,"[Cooking Table, Ground Storage, Oven, Shelves,...","[Apples, Blueberries, Carrot, Firewood, Flour,...","[Apple Pie, Blueberry Pie, Bread, Carrot Pie, ..."


In [6]:
# Zone depends on profession, consumes, and uses
# Zone Dependency:
# - Uses (furniture) dependency
# - Profession
# - Production minus consumes dependency (to include zones that produce resources that they also consume)
# Furniture dependency
# - Builder Profession
# - Resources (resources depend on which zones have been unlocked)

In [7]:
unlocked_furnitures = list(furnitures_tb[furnitures_tb["Builder Profession"].isna()]['Items'])
unlocked_professions = ["Foraging",  "Fishing", "Farming"]
unlocked_zones = list(zones_tb[zones_tb.Profession.isin(unlocked_professions)]["Zone Name"])

unlocked_resources = zones_tb[zones_tb["Zone Name"].isin(unlocked_zones)]['Produces'].to_list() + zones_tb[zones_tb["Zone Name"].isin(unlocked_zones)]['Consumes'].to_list()
unlocked_resources = list(np.unique(sum(unlocked_resources, []) + ["Ingredients"]))

#Unlock furnitures based on unlocked resources and builder profession
for row in furnitures_tb.itertuples():
    idx, item, tp, resources, quantity, profession = list(row)
    unlock = True
    for resource in resources:
        if profession:
            if profession not in unlocked_professions: 
                unlock = False
                break
        if resource not in unlocked_resources:
            unlock = False
            break
    if unlock:
        unlocked_furnitures.append(item)

# Check other unlockable_zones for non_profession related zones
unlock_zones = zones_tb[zones_tb["Profession"] == "None"]["Zone Name"].tolist()
for idx, unlock_zone in enumerate(unlock_zones):
    uses_furniture = zones_tb[zones_tb["Zone Name"] == unlock_zone].Uses.tolist()[0]
    for furniture in uses_furniture:
        unlock = False
        if furniture in unlocked_furnitures: unlock = True
        else: 
            print(furniture)
            if furniture in ['Stone Bench', 'Stone Chair','Memorial Grave', 'Stone Grave']:
                unlock = True
            else: 
                print([idx, unlock_zone, uses_furniture])
                break
    if unlock:
        unlocked_zones.append(unlock_zone)





all_professions = list(zones_tb["Profession"])
all_zones = list(zones_tb["Zone Name"])
all_resources = list(resources_tb["Resources"])

tech_tree = [{"tech_level": 0,
               "unlock_furnitures" : unlocked_furnitures ,
               "unlock_professions" : unlocked_professions,
               "unlock_zones": unlocked_zones,
               "unlock_resources": unlocked_resources
               }]


Good Stone Bench
[2, 'Chapel', ['Bookcase', 'Flower Pot', 'Good Stone Bench', 'Good Stone Chair', 'Speaking Table', 'Standing Candle']]
Speaking Table
[3, 'Church', ['Flower Pot', 'Speaking Table', 'Standing Candle', 'Wooden Bench', 'Wooden Chair']]
Execution Display
[5, 'Execution Site', ['Execution Display', 'Fireplace', 'Public Stage', 'Wooden Bench', 'Wooden Chair']]
Jail Bench
[8, 'Jail', ['Jail Bench', 'Jail Chair', 'Shelves', 'Straw Bed']]
Fireplace
[10, 'Manor', ['Barrel', 'Cabinet', 'Cauldron', 'Chest', 'Cooking Table', 'Fireplace', 'Good Wooden Table', 'Ground Storage', 'Noble Bed', 'Nouble Double Bed', 'Shelves', 'Standing Candle', 'Upholstered Chair', 'Well', 'Wooden Block']]
Stone Bench
Stone Chair
Altar
[14, 'Shrine', ['Altar', 'Well']]
Expensive Bed
[16, 'Tavern', ['Bar', 'Barrel', 'Barrel Rails', 'Cauldron', 'Cooking Table', 'Expensive Bed', 'Fireplace', 'Good Stone Bench', 'Good Stone Chair', 'Good Wooden Bench', 'Good Wooden Chair', 'Good Wooden Table', 'Ground Storag

In [8]:
for tech_level in range(1,10):
    print(f"\n\nTech level: {tech_level}")
    unlocked_furnitures = list(np.concatenate([tech_stage["unlock_furnitures"] for tech_stage in tech_tree]))
    unlocked_professions = list(np.concatenate([tech_stage["unlock_professions"] for tech_stage in tech_tree]))
    unlocked_zones = list(np.concatenate([tech_stage["unlock_zones"] for tech_stage in tech_tree]))
    unlocked_resources = list(np.concatenate([tech_stage["unlock_resources"] for tech_stage in tech_tree]))
    locked_professions = list(set(all_professions)- set(unlocked_professions))
    locked_zones = list(set(all_zones)- set(unlocked_zones))
    locked_resources = list(set(all_resources)- set(unlocked_resources))
    #locked_professions = ["Carpentering"]
    #locked_zones = ["Builder's Guild"]

    unlock_professions = []
    unlock_zones = []
    
    unlock_resources = []

    temp_df = zones_tb[zones_tb["Profession"].isin(locked_professions)]
    temp_df = temp_df[temp_df["Zone Name"].isin(locked_zones)]

    for locked_zone in temp_df.itertuples():
        unlock_furnitures = []
        # Can profession-related zone unlock
        #print(list(locked_zone))
        uses_furniture = locked_zone[4]
        zone_name =locked_zone[2]
        locked_profession = locked_zone[3]
        consumes = locked_zone[5]
        produces = locked_zone[6]
        self_making_resource = [consumed_resource for consumed_resource in consumes if consumed_resource in produces]
        #temp_resource = list(set(consumes)-set(produces))
        
        candidate_furnitures = list(set(furnitures_tb['Items'].tolist()) - set(unlocked_furnitures))

        # Which furnitures can be unlocked immediately upon unlocking a profession?
        for row in furnitures_tb[furnitures_tb['Items'].isin(candidate_furnitures)].itertuples():
            idx, item, tp, resources, quantity, profession = list(row)
            unlock = True
            if profession:
                temp_professions = unlocked_professions+[locked_profession]
                if profession not in temp_professions:
                    unlock = False
            if unlock:
                for resource in resources:
                    if resource not in unlocked_resources+self_making_resource:
                        #if resource not in self_making_resource:
                        unlock = False
                        break
            if unlock:
                #print([zone_name, locked_profession, unlock, item, uses_furniture])
                unlock_furnitures.append(item)
        #print(unlock_furnitures)

        # Can all the furnitures in the zone be unlocked?
        unlock = True
        while unlock==True:
            for furniture in uses_furniture:
                
                temp_unlocked_furnitures = unlocked_furnitures+unlock_furnitures
                if furniture in temp_unlocked_furnitures: continue
                else: 
                    if furniture in ['Stone Bench', 'Stone Chair','Memorial Grave', 'Stone Grave']:
                        continue
                    else: 
                        print([idx, zone_name, furniture, uses_furniture, unlock_furnitures])
                        unlock = False
                        
                        #print(temp_unlocked_furnitures)
                        break

            # Are consumed resources in the zone already unlocked?
            unlocked_consume = []
            not_unlocked_consume = []
            
            for resource in consumes:
                
                if resource in unlocked_resources+self_making_resource: 
                    unlocked_consume.append(resource)
                else: 
                    #print(resource)
                    not_unlocked_consume.append(resource)
                    unlock = False
                    break
            break

        if unlock:    
            print(["Unlock", locked_profession, zone_name, unlocked_consume])
            if locked_profession != "None": unlock_professions.append(locked_profession)
            unlock_zones.append(zone_name)
        if not unlock: 
            print(self_making_resource)
            print(["-----Locked", locked_profession, zone_name, not_unlocked_consume])

    # Unlock resources based on unlockable profession-related zones
    unlock_resources = zones_tb[zones_tb["Zone Name"].isin(unlock_zones)]['Produces'].to_list() + \
        zones_tb[zones_tb["Zone Name"].isin(unlock_zones)]['Consumes'].to_list()
    #print(unlock_resources)
    
    if len(str(unlock_resources))>5:
        unlock_resources = list(np.unique(np.concatenate(unlock_resources)))
        unlock_resources = [resource for resource in unlock_resources if resource in locked_resources]
    else:
        unlock_resources = []

    #print(unlock_resources)



    # Unlock furnitures based on unlockable resources
    candidate_furnitures = list(set(furnitures_tb['Items'].tolist()) - set(unlocked_furnitures))
    unlock_furnitures = []
    
    for row in furnitures_tb[furnitures_tb['Items'].isin(candidate_furnitures)].itertuples():
        idx, item, tp, resources, quantity, profession = list(row)
        unlock = True
        while unlock==True:
            if profession:
                if profession not in unlocked_professions+unlock_professions: 
                    unlock = False
            for resource in resources:
                if resource not in unlock_resources+unlocked_resources:
                    unlock = False
            break
        if unlock:
            unlock_furnitures.append(item)

    new_tech_stage = {"tech_level": tech_level,
               "unlock_furnitures" : unlock_furnitures ,
               "unlock_professions" : unlock_professions,
               "unlock_zones": unlock_zones,
               "unlock_resources": unlock_resources
               }
    print(new_tech_stage)
    tech_tree.append(new_tech_stage)
    




Tech level: 1
[165, 'Theatre', 'Theatre Stage', ['Theatre Stage', 'Wooden Bench'], []]
['None']
['-----Locked', 'Acting', 'Theatre', []]
[165, 'Alchemists Workshop', "Alchemist's Workshop", ["Alchemist's Workshop", 'Cauldron', 'Chest', 'Ground Storage', 'Oven', 'Shelves', 'Store Front', 'Well', 'Wooden Block'], []]
['Magic Potion', 'Transmutation Potion']
['-----Locked', 'Alchemy', 'Alchemists Workshop', ['Dandellion']]
[165, 'Builders Guild', "Builder's Desk", ["Builder's Desk", 'Chest', 'Ground Storage', 'Store Front', 'Well'], ['Oven']]
[]
['-----Locked', 'Architecture', 'Builders Guild', ['Stone Block']]
[165, 'Artists Workshop', 'Art Station', ['Art Station', 'Blast Furnace', 'Chest', 'Ground Storage', 'Sculpting Station', 'Store Front', 'Wooden Block'], []]
[]
['-----Locked', 'Art', 'Artists Workshop', ['Bronze Ingot']]
[165, 'Bakery', 'Oven', ['Cooking Table', 'Ground Storage', 'Oven', 'Shelves', 'Store Front', 'Well', 'Wooden Block'], []]
[]
['-----Locked', 'Baking', 'Bakery'

In [9]:
for i in tech_tree:
    print("\n".join([str(v) for v in i.values()]))
    print("\n")

0
['Hay Wall', 'Cob Wall', 'Wattle Wall', 'Wattle Fence', 'Wooden Door', 'Wooden Bridge', 'Dirt Path', 'Grass Field', 'Thatched Roof', 'Plaited Roof', 'Bar', 'Barrel', 'Barrel Rails', 'Beehive', 'Bloomery', 'Bookcase', 'Bronze Statue', 'Cabinet', 'Campfire', 'Candle', 'Caravan Storage', 'Cauldron', 'Chest', 'Chicken Bed', 'Churn', 'Cooking Table', 'Farming Soil', 'Fishing Net', 'Fishing Rod', 'Flailing Spot', 'Flower Pot', 'Golden Statue', 'Ground Storage', 'Guard Post', 'Homegrowing Pot', 'Honey Extractor', 'Jack-O-Lantern', 'Knackery', 'Outdoor Feeder', 'Painting', 'Saw Horse', 'Scraping Log', 'Shelves', 'Silo', 'Silver Statue', 'Standing Candle', 'Stone Statue', 'Store Front', 'Straw Bed', 'Straw Double Bed', 'Torch', 'Training Dummy', 'Trap', 'Tree', 'Trellis', 'Weapon Rack', 'Well', 'Wooden Bench', 'Wooden Block', 'Wooden Chair', 'Wooden Grave', 'Wooden Table', 'Wooden Table 2x2']
['Foraging', 'Fishing', 'Farming']
['Farmland', 'Fishing Pool', 'Royal Forest', 'Barracks', 'Caravan 

In [10]:
# Find zone requirements to unlock each zoned
tech_tree_dependency = [{"zone": "Farmland",
        "zone_dependency": []},
        {"zone": "Royal Forest",
        "zone_dependency": []},
        {"zone": "Fishing Pool",
        "zone_dependency": []}
        ]
tech_tree_dependency_tb = pd.DataFrame(tech_tree_dependency)
def condition_check(input: list, resource: str):
    if resource in input: return True
    else: return False

for tech_stage in tech_tree[1:]: # Skip finding the zone requirements for zones that are unlocked at the beginning
    print(tech_stage)
    for zone in tech_stage['unlock_zones']:
        zone_tb_row = zones_tb[zones_tb["Zone Name"] == zone].reset_index(drop = True).to_dict(orient = "records")[0].values()
        timestamp, zone_name, profession, uses, consumes, produces = zone_tb_row
        print(zone)
        zone_dependency = []
        
        required_resources = list([consumes])+ sum([furnitures_tb.loc[furnitures_tb["Items"]==furniture, ]["Resources"].to_list() for furniture in uses],[])
        required_profession = set(sum([furnitures_tb.loc[furnitures_tb["Items"]==furniture, ]["Builder Profession"].to_list() for furniture in uses],[]))
        required_resources = list(set(sum(required_resources,[])))

        for required_resource in required_resources:
            if required_resource =="None": pass
            else:
                output = zones_tb[zones_tb['Profession'] != "None"].loc[zones_tb.Produces.apply(lambda x: condition_check(x, required_resource)),]['Zone Name'].to_list()
                zone_dependency+=output
        for profession in required_profession:
            if profession !="None": 
                output = zones_tb.loc[zones_tb['Profession'] == profession,]['Zone Name'].to_list()
                zone_dependency+=output


            
        
        # zone_dependency = list(np.unique(zone_dependency))
        # final_zone_dependency = zone_dependency
        # for dependent_zone in zone_dependency:
        #     initial_zone_dependency = final_zone_dependency
        #     for tech_lvl in list(range(tech_stage['tech_level']-1))[::-1]:
        #         if dependent_zone in tech_tree[tech_level]
        #     if dependent_zone in tech_tree_dependency

        #     final_zone_dependency = 
        zone_dependency = list(np.unique(zone_dependency))

        if zone in zone_dependency:
            zone_dependency.remove(zone)
            
        redundant_dependencies = []
        dependency_does_not_exist = []
        condition = True

        if tech_stage['tech_level']>1:
            for repetition in range(tech_level):
                while len(zone_dependency)>0 and condition == True:
                    initial = len(zone_dependency)
                    for dependent_zone in zone_dependency:
                        if dependent_zone in tech_tree_dependency_tb.zone.to_list(): 
                            redundant_dependencies += tech_tree_dependency_tb.loc[tech_tree_dependency_tb.zone==dependent_zone,]['zone_dependency'].to_list()[0]
                            for redundant_dependency in redundant_dependencies:
                                redundant_dependencies += tech_tree_dependency_tb.loc[tech_tree_dependency_tb.zone==redundant_dependency,]['zone_dependency'].to_list()[0]
                        else:
                            dependency_does_not_exist.append(dependent_zone)
                    redundant_dependencies = list(np.unique(redundant_dependencies))
                    for redundant_dependency in list(np.unique(redundant_dependencies))+ dependency_does_not_exist:
                        if redundant_dependency in zone_dependency:
                            zone_dependency.remove(redundant_dependency)
                    final = len(zone_dependency)
                    if initial == final: condition = False
                        
        
        print(['Redundant: ', redundant_dependencies])
        print(['Dependency: ', zone_dependency])
        print(['Dependency does not exist: ', dependency_does_not_exist])
        print(list(set(required_resources)))
        
        print("\n\n")

        dependency = {"zone": zone,
        "zone_dependency": zone_dependency}

        tech_tree_dependency.append(dependency)
        tech_tree_dependency_tb = pd.DataFrame(tech_tree_dependency)

        

        
    





{'tech_level': 1, 'unlock_furnitures': ['Wooden Wall', 'Wooden Fence', 'Heavy Door', 'Wooden Fence Gate', 'Oak Wood Floor', 'Brich Wood Floor', 'Maple Wood Floor', 'Poplar Wood Floor', 'Spruce Wood Floor', 'Board Roof', 'Apothecary Table', 'Art Station', "Bowyer's Workshop", 'Brewing Station', "Builder's Desk", "Butcher's Table", 'Filtering Tub', 'Forest Feeder', 'Good Wooden Bench', 'Good Wooden Chair', 'Grape Mashing Tub', 'Hideout', 'Jail Bench', 'Jail Chair', 'Leatherworking Table', 'Loom', 'Meat Hooks', 'Paper Mill', 'Public Stage', 'Rope Walk', 'Speaking Table', 'Spinning Wheel', 'Stretching Frame', 'Tailoring Workshop', 'Tanning Log', 'Tanning Pit', 'Wooden Bed', 'Wooden Double Bed', 'Writing Table'], 'unlock_professions': ['Beekeeping', 'Carpentering', 'Graveyardkeeping', 'Mining'], 'unlock_zones': ['Apiary', 'Carpentry', 'Graveyard', 'Mining Field'], 'unlock_resources': ['Basic Repair Kit', 'Coal', 'Copper Ore', 'Gem Ore', 'Gold Ore', 'Honey', 'Iron ore', 'Limestone', 'Silver 

In [11]:
list(range(tech_stage['tech_level']-1))[::-1]

[7, 6, 5, 4, 3, 2, 1, 0]

In [12]:
tech_tree_dependency_tb

,zone,zone_dependency
0,Farmland,[]
1,Royal Forest,[]
2,Fishing Pool,[]
3,Apiary,"[Farmland, Royal Forest]"
4,Carpentry,[Royal Forest]
5,Graveyard,"[Builders Guild, Mining Field, Royal Forest]"
6,Mining Field,[Royal Forest]
7,Brewery,"[Carpentry, Farmland, Mining Field]"
8,Butchery,"[Carpentry, Farmland]"
9,Hunting Area,[Carpentry]
